# Summary

---

Convert archive files downloaded using the `download.ipynb` notebook to HDF5 file format.

# Imports

In [1]:
%run _imports.ipynb

Setting the PYTHON_VERSION environment variable.
Setting the SPARK_MASTER environment variable.
Setting the DB_TYPE environment variable.
Setting the DB_PORT environment variable.


2017-12-18 14:07:52.007577


In [2]:
import zipfile

In [3]:
import tables
import xlrd

In [4]:
import warnings
warnings.filterwarnings("ignore", category=tables.NaturalNameWarning)

In [166]:
import kmtools.db_tools

In [5]:
import pmc_tables

importlib.reload(pmc_tables)

<module 'pmc_tables' from '/home/kimlab1/strokach/working/pmc_tables/pmc_tables/__init__.py'>

In [6]:
NOTEBOOK_NAME = 'archive_to_hdf5'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

# Parameters

In [7]:
ARCHIVE_PATH = Path(f"download/pmc_archives").absolute()
ARCHIVE_PATH

PosixPath('/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives')

In [8]:
OUTPUT_PATH = Path(f"{NOTEBOOK_NAME}/pmc_archives").absolute()
OUTPUT_PATH

PosixPath('/kimlab2/database_data/datapkg/pmc-tables/notebooks/archive_to_hdf5/pmc_archives')

# Input

In [9]:
downloaded_df = pd.read_csv('download/downloaded.csv.gz')

In [10]:
downloaded_df.head()

,pmc_type,pmc_id,source_url,archive_file_path,output_file_path,is_downloaded
0,ncbi_oa_package,PMC13900,ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/08/e0/PMC13900.tar.gz,oa_package/08/e0/PMC13900.tar.gz,b5/c0/PMC13900.zip,True
1,ncbi_oa_package,PMC13901,ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/b0/ac/PMC13901.tar.gz,oa_package/b0/ac/PMC13901.tar.gz,39/28/PMC13901.zip,True
2,ncbi_oa_package,PMC13902,ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/f7/98/PMC13902.tar.gz,oa_package/f7/98/PMC13902.tar.gz,04/bc/PMC13902.zip,True
3,ncbi_oa_package,PMC13911,ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/9c/7f/PMC13911.tar.gz,oa_package/9c/7f/PMC13911.tar.gz,69/8a/PMC13911.zip,True
4,ncbi_oa_package,PMC13912,ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/c6/fb/PMC13912.tar.gz,oa_package/c6/fb/PMC13912.tar.gz,ed/e5/PMC13912.zip,True


# Functions

# Workflow

In [100]:
ERRORS_PATH = Path(NOTEBOOK_NAME).joinpath('errors_1')
ERRORS_PATH

PosixPath('archive_to_hdf5/errors_1')

In [101]:
ERRORS = []

In [102]:
logging.getLogger('pmc_tables').setLevel(logging.DEBUG)

In [103]:
import pmc_tables

importlib.reload(pmc_tables)

<module 'pmc_tables' from '/home/kimlab1/strokach/working/pmc_tables/pmc_tables/__init__.py'>

In [104]:
start_row = 0
num_rows = 1000

with pd.HDFStore(f"{NOTEBOOK_NAME}/test_3.h5", mode='w', compression='blosc:blosclz', complevel=6) as store:
    for row in tqdm.tqdm_notebook(
            itertools.islice(downloaded_df.itertuples(), start_row, start_row + num_rows),
            total=num_rows):
        archive_file = ARCHIVE_PATH.joinpath(row.output_file_path)
        with zipfile.ZipFile(archive_file) as archive:
            try:
                pmc_tables.save_archive_to_hdf5(row.pmc_id, archive, store)
            except (TypeError, ValueError, xlrd.XLRDError, pmc_tables.ParserError) as e:
                print(f"{type(e)}: {e}".replace('\n', ' '))
#                 raise e

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

/home/kimlab1/strokach/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py:809: FutureWarning: The 'tupleize_cols' argument has been deprecated and will be removed in a future version. Column tuples will then always be converted to MultiIndex.


  self.options, self.engine = self._clean_options(options, engine)
Encountered error `Cannot serialize the column [Concentrations of protein in each Cy5 mix (1=highest, 6=lowest)] because
its data contents are [mixed-integer] object dtype`
Converting column `('FRat2 (Red/Green Ratios)', nan)` with values `['Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR'
 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-Adaptin alpha']` to `str`.
Converting column `('Data reordered by concentration instead of array', 'Ideal')` with values `['Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR'
 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-Adaptin alpha']` to `str`.
Converting column `('Dil. 1', 1.505149978319906)` with values `[1.24194574270

Converting column `('_16', 'Dil. 6', -1.505149978319906)` with values `[-1.228403089273344 -0.9731305841700381 -0.18701040411511893
 -1.3253131022814002 -1.6263430979453815 -1.6263430979453815
 -1.4502518388897006 -1.0242831066174194 -0.17918506660316236
 1.0126450613983005]` to `str`.
Converting column `('_23', 'Ideal', nan)` with values `['Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR'
 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-Actin']` to `str`.
Converting column `('_49', 'NAME', 'Ideal')` with values `['Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR'
 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-Actin']` to `str`.
Converting column `('_55', nan, nan)` with values `['Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR'
 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-Actin']` to `str`.
Converting column `('_67', nan, nan)` with values `['Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-ABR'
 'Anti-ABR' 'Anti-ABR' 'Anti-ABR' 'Anti-Actin']` to `

Converting column `_3` with values `[nan 'Treatments' nan
 'Patients had application of either honey or antiseptic onto a wound freshly washed with saline every 12 hours. The wound site was then covered with a dry gauze.  In addition all patients received systemic antibiotics depending on wound cultures and sensitivity testing.\nHoney group n=26\nAntiseptic group n=24\nHoney samples were Yemeni honey from villages, cultured to exclude bacterial contamination'
 'Honey group (n=25): Honey was applied to the wound and then was covered with sterile gauze and bandaged. Honey was applied on alternate days. The wounds were washed with normal saline before application of dressings. Skin grafts were applied as needed.\nTangential excision (TE) group (n=25): Operation was performed 3-6 days after burn. All patients had a skin graft by day 6. Dressings were removed on the 5th day after the operation to assess the take of the graft.\nHoney was unprocessed and undiluted and straight from the hive'


Converting column `_9` with values `[nan 'OPVS' nan 8 5 7 10 8 10 10]` to `str`.
Converting column `_10` with values `[nan 'Author \nconclusion' nan 'Positive' 'Negative' 'Positive' 'Positive'
 'Positive' 'Positive' 'Positive']` to `str`.
Converting column `_11` with values `[nan 'Reviewer\nconclusion' nan 'Positive' 'Negative' 'Positive' 'Positive'
 'Positive' 'Positive' 'Positive']` to `str`.
Encountered error `Cannot serialize the column [_2] because
its data contents are [mixed-integer] object dtype`
Converting column `('Additional file 3: Results and calculations for individual studies', 'Trial')` with values `[nan 'Taylor 75' 'Ransden 77' 'Spigno' 'Hughes' 'Porru' 'Bonci' 'Commins'
 'Combined']` to `str`.
Converting column `('_1', 'Children    ears')` with values `[nan 'ears' 'children' 'ears' 'children' 'ears' 'children' 'children' nan]` to `str`.
Converting column `('_4', 'ITT/\nper protocol')` with values `[nan 'ITT' 'PP' 'ITT' 'PP' 'PP' 'ITT' 'ITT' nan]` to `str`.
Encountered

<class 'pmc_tables.errors.ParserError'>: Could not parse file: /tmp/strokach/tmp8oqzjfw6/gb-2001-2-10-research0041-S2.csv.


Encountered error `Cannot serialize the column [] because
its data contents are [mixed-integer] object dtype`
Converting column `Species` with values `[nan nan nan 'T. thermophila' 'T. malaccensis' 'T. mobilis' 'T. borealis'
 'T. rostrata' 'T. setosa' 'T. vorax']` to `str`.
Converting column `` with values `[nan 129 nan 'G' '.' '.' '.' '.' '.' '.']` to `str`.
Converting column `_1` with values `[nan 189 nan 'C' '.' '.' '.' '.' 'T' 'T']` to `str`.
Converting column `_2` with values `[nan 216 nan 'G' '.' '.' '.' '.' '.' '.']` to `str`.
Converting column `_3` with values `[nan 221 nan 'A' '.' '.' '.' '.' '.' 'G']` to `str`.
Converting column `_4` with values `[nan 228 nan 'C' '.' '.' '.' '.' '.' 'T']` to `str`.
Converting column `_5` with values `[nan 262 nan 'G' '.' 'A' 'A' 'A' 'A' 'A']` to `str`.
Converting column `_6` with values `[nan 267 nan 'A' '.' '.' '.' '.' '.' '.']` to `str`.
Converting column `_7` with values `[nan 268 nan 'G' '.' '.' 'A' 'A' 'A' 'A']` to `str`.
Converting colu

Converting column `_2` with values `[nan '1st base of exon=codon' 2 1 1 1 1 3 2 1]` to `str`.
Converting column `_3` with values `[nan 'exon size' 62 93 78 93 173 119 182 129]` to `str`.
Converting column `_4` with values `[nan 'last base of exon=codon' 3 3 3 3 2 1 3 3]` to `str`.
Converting column `_5` with values `[nan 'Fragment size' 233 384 322 268 356 350 494 321]` to `str`.


<class 'ValueError'>: No tables found matching pattern '.+'
<class 'ValueError'>: No tables found matching pattern '.+'
<class 'xlrd.biffh.XLRDError'>: Unsupported format, or corrupt file: Expected BOF record; found b'Informat'


Encountered error `Cannot serialize the column [] because
its data contents are [mixed-integer] object dtype`
Converting column `2001-07-25 00:00:00` with values `[nan
 'This workbook contains supplemental data for the Hill et al. manuscript.'
 nan
 'Each worksheet in the book contains experimental data for one of the three'
 'array designs (A, B, or C), in one of the four metrics (AD, ADs, F, Fs)'
 'The GeneChip Absolute Decisions (A=absent, M=marginal, P=present) are also given for each hybridization '
 'in the sheets "a_abs_dec", "b_abs_dec", "c_abs_dec"' nan
 'The first column of each worksheet contains the Qualifer names corresponding to each transcript'
 nan]` to `str`.
Converting column `` with values `[nan nan nan nan nan nan nan nan nan nan]` to `str`.
Converting column `_1` with values `[nan nan nan nan nan nan nan nan nan nan]` to `str`.
Converting column `_2` with values `[nan nan nan nan nan nan nan nan nan nan]` to `str`.
Encountered error `Cannot serialize the column [4]

In [108]:
len(ERRORS)

4

In [109]:
df = pd.DataFrame(ERRORS)

In [110]:
df.head()

,0,1,2,3,4,5,6
0,/PMC59832/1471-2156-2-17-S1.xls/sheet_1,<class 'TypeError'>,Cannot serialize the column [level_1] because\nits data contents are [mixed-integer] object dtype,/1471-2156-2-17-S1.xls/sheet_1,"{'label': 'Sheet2', 'parser_name': 'excel_parser', 'status': 'error', 'fixers_applied': ['fix_column_names', 'fix_ex...",Primer sequences and DHPLC temperatures Excel Spreadsheet Unnamed...,archive_to_hdf5/errors_1/PMC59832/1471-2156-2-17-S1.xls
1,/PMC64840/gb-2001-2-12-research0055-S1.xls/sheet_5,<class 'TypeError'>,"Passing an incorrect value to a table column. Expected a Col (or subclass) instance and got: ""ObjectAtom()"". Please ...",/gb-2001-2-12-research0055-S1.xls/sheet_5,"{'label': 'a_abs_dec', 'parser_name': 'excel_parser', 'status': 'error', 'fixers_applied': ['fix_column_names', 'fix...",4 10 20 21 5 6 1 19 7 17 18 8 11 12 13 9 14 15 16 ...,archive_to_hdf5/errors_1/PMC64840/gb-2001-2-12-research0055-S1.xls
2,/PMC64840/gb-2001-2-12-research0055-S1.xls/sheet_10,<class 'TypeError'>,"Passing an incorrect value to a table column. Expected a Col (or subclass) instance and got: ""ObjectAtom()"". Please ...",/gb-2001-2-12-research0055-S1.xls/sheet_10,"{'label': 'b_abs_dec', 'parser_name': 'excel_parser', 'status': 'error', 'fixers_applied': ['fix_column_names', 'fix...",5 8 10 14 6 11 2 4 7 12 13 15 9 1 3 NaN ...,archive_to_hdf5/errors_1/PMC64840/gb-2001-2-12-research0055-S1.xls
3,/PMC64840/gb-2001-2-12-research0055-S1.xls/sheet_15,<class 'TypeError'>,"Passing an incorrect value to a table column. Expected a Col (or subclass) instance and got: ""ObjectAtom()"". Please ...",/gb-2001-2-12-research0055-S1.xls/sheet_15,"{'label': 'c_abs_dec', 'parser_name': 'excel_parser', 'status': 'error', 'fixers_applied': ['fix_column_names', 'fix...",5 6 7 12 8 13 1 4 9 14 10 15 11 16 2 3 Na...,archive_to_hdf5/errors_1/PMC64840/gb-2001-2-12-research0055-S1.xls


In [111]:
ERRORS[0][:5]

('/PMC59832/1471-2156-2-17-S1.xls/sheet_1',
 TypeError,
 TypeError('Cannot serialize the column [level_1] because\nits data contents are [mixed-integer] object dtype'),
 '/1471-2156-2-17-S1.xls/sheet_1',
 {'error_message': 'Cannot serialize the column [_2] because\nits data contents are [mixed-integer] object dtype',
  'error_message_2': 'Cannot serialize the column [level_1] because\nits data contents are [mixed-integer] object dtype',
  'final_error': "<class 'TypeError'>",
  'final_error_message': 'Cannot serialize the column [level_1] because\nits data contents are [mixed-integer] object dtype',
  'fixers_applied': ['fix_column_names',
   'fix_extra_headers_and_footers',
   'fix_column_dtypes',
   'fix_column_names'],
  'label': 'Sheet2',
  'parser_name': 'excel_parser',
  'status': 'error'})

In [113]:
df1 = ERRORS[0][6]

# Run jobs

## Job parameters

In [129]:
JOB_ID = 'job_0'
DATA_ID = 'pmc_archives_0'

In [206]:
JOB_DIR = f"~/datapkg/{os.environ['DB_SCHEMA']}/notebooks/{NOTEBOOK_NAME}/{JOB_ID}"
DATA_DIR = f"~/datapkg/{os.environ['DB_SCHEMA']}/notebooks/{NOTEBOOK_NAME}/{DATA_ID}"

os.makedirs(op.expanduser(JOB_DIR), exist_ok=True)
os.makedirs(op.expanduser(DATA_DIR), exist_ok=True)
print(JOB_DIR, DATA_DIR, sep='\n')

~/datapkg/pmc-tables/notebooks/archive_to_hdf5/job_0
~/datapkg/pmc-tables/notebooks/archive_to_hdf5/pmc_archives_0


In [207]:
ENV = {
    'PATH': '/home/kimlab1/strokach/anaconda/bin:/usr/local/bin:/usr/bin:/bin',
    'OMP_NUM_THREADS': '1',
    'OPENMM_CPU_THREADS': '1',
}

## Script file

In [167]:
with Path(f'{DATA_DIR}.py').expanduser().open() as fin:
    code = fin.read()

kmtools.db_tools.print_sql(code, 'python')

## System commands

In [268]:
system_commands = []

for input_dir in ARCHIVE_PATH.glob('??'):
    output_file = Path(DATA_DIR).expanduser().joinpath(input_dir.name).with_suffix('.h5')
    if input_dir.parts[1] != 'home':
        input_dir = Path('/home').joinpath(*input_dir.parts[1:])
    if output_file.parts[1] != 'home':
        output_file = Path('/home').joinpath(*output_file.parts[1:])
    system_command = f'python "../{DATA_ID}.py" -i "{input_dir}" -o "{output_file}"'
    system_commands.append(system_command)
    
system_commands = pd.DataFrame(system_commands, columns=['system_command'])

In [269]:
system_commands[:10]

,system_command
0,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/cd""..."
1,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/29""..."
2,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/f3""..."
3,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/87""..."
4,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/37""..."
5,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/66""..."
6,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/47""..."
7,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/be""..."
8,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/a3""..."
9,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/b2""..."


In [272]:
!(cd {DATA_DIR}; echo {system_commands.iat[0, 0]})

python ../pmc_archives_0.py -i /home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/cd -o /home/kimlab1/strokach/datapkg/pmc-tables/notebooks/archive_to_hdf5/pmc_archives_0/cd.h5


In [309]:
!(cd {DATA_DIR}; {system_commands.iat[0, 0]})

[__main__] Counter: 0
[pmc_tables.pmc_tables] pmc_id: `PMC3541495`
[pmc_tables.pmc_tables] pmc_id: `PMC3286889`
/home/kimlab1/strokach/anaconda/lib/python3.6/site-packages/pandas/io/parsers.py:809: FutureWarning: The 'tupleize_cols' argument has been deprecated and will be removed in a future version. Column tuples will then always be converted to MultiIndex.


  self.options, self.engine = self._clean_options(options, engine)
[pmc_tables.pmc_tables] pmc_id: `PMC2192392`
[pmc_tables.pmc_tables] pmc_id: `PMC3657259`
[pmc_tables.pmc_tables] pmc_id: `PMC4109565`
[pmc_tables.pmc_tables] pmc_id: `PMC4388329`
[pmc_tables.pmc_tables] pmc_id: `PMC3754230`
[pmc_tables.pmc_tables] pmc_id: `PMC5609134`
[pmc_tables.pmc_tables] pmc_id: `PMC3604601`
[pmc_tables.pmc_tables] pmc_id: `PMC3137854`
[pmc_tables.pmc_tables] pmc_id: `PMC3332603`
[pmc_tables.pmc_tables] pmc_id: `PMC4745324`
[pmc_tables.pmc_tables] pmc_id: `PMC151679`
[pmc_tables.pmc_tables] pmc_id: `PMC3793721`
[pmc_tables.parsers.xml] Unexp

[pmc_tables.pmc_tables] pmc_id: `PMC5240672`
[pmc_tables.pmc_tables] pmc_id: `PMC2010836`
[pmc_tables.pmc_tables] pmc_id: `PMC2480508`
[pmc_tables.pmc_tables] pmc_id: `PMC2225460`
[pmc_tables.pmc_tables] pmc_id: `PMC3239560`
[pmc_tables.pmc_tables] pmc_id: `PMC4530061`
[pmc_tables.pmc_tables] pmc_id: `PMC2897079`
[pmc_tables.pmc_tables] pmc_id: `PMC3678587`
[pmc_tables.pmc_tables] pmc_id: `PMC4863118`
[pmc_tables.pmc_tables] pmc_id: `PMC4083992`
[pmc_tables.pmc_tables] pmc_id: `PMC4168840`
[pmc_tables.parsers.xml] Unexpected table `attrib_set`: {'content-type'}
[pmc_tables.parsers.xml] Unexpected table `attrib_set`: {'content-type'}
[pmc_tables.pmc_tables] pmc_id: `PMC2819791`
[pmc_tables.pmc_tables] pmc_id: `PMC3701221`
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.pmc_tables] pmc_id: `PMC5094980`
[pmc_tables.pmc_tables] pmc_id: `PMC2564113`
[pmc_tables.pmc_tables] pmc_id: `PMC3961341`
[pmc_tables.pmc_tables] pmc_id: `PMC4045875`
[pmc_tables.pm

[pmc_tables.pmc_tables] pmc_id: `PMC5582868`
[pmc_tables.pmc_tables] pmc_id: `PMC4233003`
[pmc_tables.pmc_tables] pmc_id: `PMC5013386`
[pmc_tables.pmc_tables] pmc_id: `PMC5677304`
[pmc_tables.pmc_tables] pmc_id: `PMC4492741`
[pmc_tables.pmc_tables] pmc_id: `PMC2923414`
[pmc_tables.pmc_tables] pmc_id: `PMC2908010`
[pmc_tables.pmc_tables] pmc_id: `PMC5160159`
[pmc_tables.pmc_tables] pmc_id: `PMC3970218`
[pmc_tables.pmc_tables] pmc_id: `PMC3838601`
[pmc_tables.pmc_tables] pmc_id: `PMC5424766`
[pmc_tables.pmc_tables] pmc_id: `PMC4730589`
[pmc_tables.pmc_tables] pmc_id: `PMC2997797`
[pmc_tables.pmc_tables] pmc_id: `PMC4322441`
[pmc_tables.pmc_tables] pmc_id: `PMC3666883`
[pmc_tables.pmc_tables] pmc_id: `PMC4128553`
[pmc_tables.pmc_tables] pmc_id: `PMC5358217`
[pmc_tables.pmc_tables] pmc_id: `PMC4869688`
[pmc_tables.pmc_tables] pmc_id: `PMC4509848`
[pmc_tables.pmc_tables] pmc_id: `PMC5428136`
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'alternatives'}
[pmc_tables.pmc_tabl

[pmc_tables.pmc_tables] pmc_id: `PMC4258516`
[pmc_tables.pmc_tables] pmc_id: `PMC5656884`
[pmc_tables.pmc_tables] pmc_id: `PMC1637733`
[pmc_tables.pmc_tables] pmc_id: `PMC4433343`
[pmc_tables.pmc_tables] pmc_id: `PMC3058637`
[pmc_tables.pmc_tables] pmc_id: `PMC4796972`
[pmc_tables.pmc_tables] pmc_id: `PMC4811345`
[pmc_tables.pmc_tables] pmc_id: `PMC3793015`
[pmc_tables.pmc_tables] pmc_id: `PMC2914259`
[pmc_tables.pmc_tables] pmc_id: `PMC4521769`
[pmc_tables.pmc_tables] pmc_id: `PMC4232342`
[pmc_tables.pmc_tables] pmc_id: `PMC4916292`
[pmc_tables.pmc_tables] pmc_id: `PMC3391610`
[pmc_tables.pmc_tables] pmc_id: `PMC5061872`
[pmc_tables.pmc_tables] pmc_id: `PMC2294123`
[pmc_tables.pmc_tables] pmc_id: `PMC4106153`
[pmc_tables.pmc_tables] pmc_id: `PMC3741145`
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'object-id', 'alternatives'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'object-id', 'alternatives'}
[pmc_tables.parsers.xml] Unexpected table-wrap `c

[pmc_tables.pmc_tables] pmc_id: `PMC4898419`
[pmc_tables.pmc_tables] pmc_id: `PMC4263435`
[pmc_tables.pmc_tables] pmc_id: `PMC4985760`
[pmc_tables.pmc_tables] pmc_id: `PMC2124939`
[pmc_tables.pmc_tables] pmc_id: `PMC4309372`
[pmc_tables.pmc_tables] pmc_id: `PMC4491415`
[pmc_tables.pmc_tables] pmc_id: `PMC4491151`
[pmc_tables.pmc_tables] pmc_id: `PMC4699217`
[pmc_tables.pmc_tables] pmc_id: `PMC1719213`
[pmc_tables.pmc_tables] pmc_id: `PMC5432806`
[pmc_tables.pmc_tables] pmc_id: `PMC4617474`
[pmc_tables.pmc_tables] pmc_id: `PMC5175394`
[pmc_tables.pmc_tables] pmc_id: `PMC4537571`
^C
put_cell 20512 0
Traceback (most recent call last):
  File "/home/kimlab1/strokach/anaconda/lib/python3.6/site-packages/xlrd/sheet.py", line 698, in put_cell_unragged
    self._cell_types[rowx][colx] = ctype
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "../pmc_archives_0.py", line 41, in <module>
    main(zi

## Submit

In [395]:
system_commands_2 = pd.concat([system_commands, system_commands], ignore_index=True)

In [398]:
display(system_commands_2.tail())
print(len(system_commands_2) / 2)

,system_command
507,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/a5""..."
508,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/ca""..."
509,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/59""..."
510,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/da""..."
511,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/a4""..."


256.0


In [399]:
import jobsubmitter

importlib.reload(jobsubmitter)

<module 'jobsubmitter' from '/home/kimlab1/strokach/working/jobsubmitter/jobsubmitter/__init__.py'>

In [400]:
js = jobsubmitter.JobSubmitter('beagle', JOB_DIR, DATA_DIR, ENV)
js

In [401]:
jo = jobsubmitter.JobOpts(JOB_ID, nproc=1, queue='medium', walltime='24:00:00', mem='12G')
jo

JobOpts(job_name='job_0', queue='medium', nproc=1, walltime='24:00:00', mem='12G', pmem=None, vmem=None, pvmem=None, email='noname@example.com', email_opts='a', qsub_shell='/bin/bash', qsub_script_args=None)

In [402]:
logging.getLogger().setLevel(logging.INFO)

In [403]:
futures = js.submit(system_commands_2[256:], jo, deplay=0.02)

HBox(children=(IntProgress(value=0, max=256), HTML(value='')))

Secsh channel 217 open FAILED: open failed: Administratively prohibited
The following exception occured:
(1, 'Administratively prohibited')
Retrying...
Secsh channel 218 open FAILED: open failed: Administratively prohibited
The following exception occured:
(1, 'Administratively prohibited')
Retrying...
Secsh channel 219 open FAILED: open failed: Administratively prohibited
The following exception occured:
(1, 'Administratively prohibited')
Retrying...
Secsh channel 220 open FAILED: open failed: Administratively prohibited
The following exception occured:
(1, 'Administratively prohibited')
Retrying...
Secsh channel 221 open FAILED: open failed: Administratively prohibited
The following exception occured:
(1, 'Administratively prohibited')
Retrying...
Secsh channel 222 open FAILED: open failed: Administratively prohibited
The following exception occured:
(1, 'Administratively prohibited')
Retrying...
Secsh channel 223 open FAILED: open failed: Administratively prohibited
The following ex

**Results:**

In [408]:
results = js.job_status(system_commands_2)

HBox(children=(IntProgress(value=0, max=512), HTML(value='')))

In [409]:
done = results[results['status'] == 'done']
print(len(done))

11


In [410]:
failed = results[results['status'] == 'error']
print(len(failed))

11


In [411]:
failed

,system_command,status,stdout_data
Index,,,
10,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/95""...",error,NaN
30,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/33""...",error,NaN
31,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/d4""...",error,NaN
32,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/5c""...",error,NaN
33,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/e9""...",error,NaN
43,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/dd""...",error,NaN
52,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/9d""...",error,NaN
61,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/1d""...",error,NaN
63,"python ""../pmc_archives_0.py"" -i ""/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/f9""...",error,NaN


**Do this in a loop until done...**

In [ ]:
futures = js.submit(failed, jo, deplay=0.02)

In [ ]:
results2 = js.job_status(system_commands)

In [ ]:
failed = results2[results2['status'] != 'done']
print(len(failed))

# Validate

In [383]:
hdf5_store = Path(f'{NOTEBOOK_NAME}/test_4.h5').absolute()
input_dir = Path('/home/kimlab2/database_data/datapkg/pmc-tables/notebooks/download/pmc_archives/29')

[autoreload of pmc_tables failed: Traceback (most recent call last):
  File "/home/kimlab1/strokach/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 246, in check
    superreload(m, reload, self.old_objects)
  File "/home/kimlab1/strokach/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 369, in superreload
    module = reload(module)
  File "/home/kimlab1/strokach/anaconda3/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/kimlab1/strokach/anaconda3/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/kimlab1/strokach/working/pmc_tables/pmc_tables/__init__.py", line 11, in <module>
    from . import *
AttributeError: module '

In [384]:
!python {DATA_DIR}.py -i {input_dir} -o {hdf5_store}

[__main__] Counter: 0
[pmc_tables.pmc_tables] pmc_id: `PMC3564939`
[pmc_tables.pmc_tables] PMC id `PMC3564939` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3101524`
[pmc_tables.pmc_tables] PMC id `PMC3101524` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4553635`
[pmc_tables.pmc_tables] PMC id `PMC4553635` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC5041728`
[pmc_tables.pmc_tables] PMC id `PMC5041728` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4483340`
[pmc_tables.pmc_tables] PMC id `PMC4483340` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC5072973`
[pmc_tables.pmc_tables] PMC id `PMC5072973` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4936977`
[pmc_tables.pmc_tables] PMC id `PMC4936977` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC5467369`
[pmc_tables.pmc_tables] PMC id `PMC5467369` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC556114

[pmc_tables.pmc_tables] PMC id `PMC3283065` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC5337901`
[pmc_tables.pmc_tables] PMC id `PMC5337901` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3346972`
[pmc_tables.pmc_tables] PMC id `PMC3346972` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4553080`
[pmc_tables.pmc_tables] PMC id `PMC4553080` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4523265`
[pmc_tables.pmc_tables] PMC id `PMC4523265` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4139492`
[pmc_tables.pmc_tables] PMC id `PMC4139492` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3980861`
[pmc_tables.pmc_tables] PMC id `PMC3980861` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC5064382`
[pmc_tables.pmc_tables] PMC id `PMC5064382` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC2173525`
[pmc_tables.pmc_tables] PMC id `PMC2173525` already exists. Skip

[pmc_tables.pmc_tables] PMC id `PMC3372972` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3691851`
[pmc_tables.pmc_tables] PMC id `PMC3691851` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3486826`
[pmc_tables.pmc_tables] PMC id `PMC3486826` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3336617`
[pmc_tables.pmc_tables] PMC id `PMC3336617` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3797864`
[pmc_tables.pmc_tables] PMC id `PMC3797864` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4054314`
[pmc_tables.pmc_tables] PMC id `PMC4054314` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3052958`
[pmc_tables.pmc_tables] PMC id `PMC3052958` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3631012`
[pmc_tables.pmc_tables] PMC id `PMC3631012` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4521295`
[pmc_tables.pmc_tables] PMC id `PMC4521295` already exists. Skip

[pmc_tables.pmc_tables] pmc_id: `PMC3946503`
[pmc_tables.pmc_tables] PMC id `PMC3946503` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC5487506`
[pmc_tables.pmc_tables] PMC id `PMC5487506` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4053641`
[pmc_tables.pmc_tables] PMC id `PMC4053641` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3922472`
[pmc_tables.pmc_tables] PMC id `PMC3922472` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3807995`
[pmc_tables.pmc_tables] PMC id `PMC3807995` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3959255`
[pmc_tables.pmc_tables] PMC id `PMC3959255` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4298470`
[pmc_tables.pmc_tables] PMC id `PMC4298470` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4630252`
[pmc_tables.pmc_tables] PMC id `PMC4630252` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3880032`
[pmc_tables.pmc_tab

[pmc_tables.pmc_tables] PMC id `PMC3043568` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC5064692`
[pmc_tables.pmc_tables] PMC id `PMC5064692` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4747756`
[pmc_tables.pmc_tables] PMC id `PMC4747756` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4745423`
[pmc_tables.pmc_tables] PMC id `PMC4745423` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4375037`
[pmc_tables.pmc_tables] PMC id `PMC4375037` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC4623199`
[pmc_tables.pmc_tables] PMC id `PMC4623199` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3657291`
[pmc_tables.pmc_tables] PMC id `PMC3657291` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC3855995`
[pmc_tables.pmc_tables] PMC id `PMC3855995` already exists. Skipping...
[pmc_tables.pmc_tables] pmc_id: `PMC2363116`
[pmc_tables.pmc_tables] PMC id `PMC2363116` already exists. Skip

...

[pmc_tables.pmc_tables] pmc_id: `PMC3970641`
[pmc_tables.pmc_tables] pmc_id: `PMC4698254`
[pmc_tables.pmc_tables] pmc_id: `PMC3916315`
[pmc_tables.pmc_tables] pmc_id: `PMC4136470`
[pmc_tables.pmc_tables] pmc_id: `PMC5552224`
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'alt-text'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'alt-text'}
[pmc_tables.pmc_tables] pmc_id: `PMC3198183`
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.pmc_tables] pmc_id: `PMC5606712`
[pmc_tables.pmc_tables] pmc_id: `PMC3600040`
[pmc_tables.pmc_tables] pmc_id: `PMC3432373`
[pmc_tables.pmc_tables] pmc_id: `PMC3174463`
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml

[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'object-id'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'object-id'}
[pmc_tables.pmc_tables] pmc_id: `PMC3150311`
[pmc_tables.pmc_tables] pmc_id: `PMC2109100`
[pmc_tables.pmc_tables] pmc_id: `PMC5510504`
[pmc_tables.pmc_tables] pmc_id: `PMC3838837`
[pmc_tables.pmc_tables] pmc_id: `PMC3037906`
[pmc_tables.pmc_tables] pmc_id: `PMC4261739`
[pmc_tables.pmc_tables] pmc_id: `PMC5587869`
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'alt-text'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'alt-text'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'alt-text'}
[pmc_tables.pmc_tables] pmc_id: `PMC2587559`
[pmc_tables.pmc_tables] pmc_id: `PMC5047706`
[pmc_tables.pmc_tables] pmc_id: `PMC2185983`
[pmc_tables.pmc_tables] pmc_id: `PMC3003855`
[pmc_tables.pmc_tables] pmc_id: `PMC5627989`
[pmc_tables.pmc_tables] pmc_id: `PMC3585355`
[pmc_tables.parsers.xml] Unexpect

[pmc_tables.pmc_tables] pmc_id: `PMC3120243`
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.pmc_tables] pmc_id: `PMC3644233`
[pmc_tables.pmc_tables] pmc_id: `PMC3118393`
[pmc_tables.pmc_tables] pmc_id: `PMC4596587`
[pmc_tables.pmc_tables] pmc_id: `PMC2856814`
[pmc_tables.pmc_tables] pmc_id: `PMC4782347`
[pmc_tables.pmc_tables] pmc_id: `PMC5565529`
[pmc_tables.pmc_tables] pmc_id: `PMC2073539`
[pmc_tables.pmc_tables] pmc_id: `PMC5536667`
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'alter

[pmc_tables.pmc_tables] pmc_id: `PMC4907099`
[pmc_tables.pmc_tables] pmc_id: `PMC3128826`
[pmc_tables.pmc_tables] pmc_id: `PMC5107608`
[pmc_tables.pmc_tables] pmc_id: `PMC4672189`
[pmc_tables.pmc_tables] pmc_id: `PMC1519035`
[pmc_tables.pmc_tables] pmc_id: `PMC5056279`
[pmc_tables.pmc_tables] pmc_id: `PMC3042486`
[pmc_tables.pmc_tables] pmc_id: `PMC3342752`
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'graphic'}
[pmc_tables.pmc_tables] pmc_id: `PMC3914970`
[pmc_tables.parsers.xml] Unexpected table-wrap `children_set`: {'object-id', 'alternatives'}
[pmc_tables.parsers.xml] Unexpected table-wrap `chil

[pmc_tables.pmc_tables] pmc_id: `PMC3376251`
[pmc_tables.pmc_tables] pmc_id: `PMC4003002`
[pmc_tables.pmc_tables] pmc_id: `PMC4062188`
[pmc_tables.pmc_tables] pmc_id: `PMC5074730`
[pmc_tables.pmc_tables] pmc_id: `PMC2720172`
[pmc_tables.pmc_tables] pmc_id: `PMC2891070`
[pmc_tables.pmc_tables] pmc_id: `PMC2691684`
[pmc_tables.pmc_tables] pmc_id: `PMC4159659`
^C
Traceback (most recent call last):
  File "/home/kimlab1/strokach/datapkg/pmc-tables/notebooks/archive_to_hdf5/pmc_archives_0.py", line 41, in <module>
    main(zip_archives, hdf5_file)
  File "/home/kimlab1/strokach/datapkg/pmc-tables/notebooks/archive_to_hdf5/pmc_archives_0.py", line 26, in main
    pmc_tables.save_archive_to_hdf5(archive, store)
  File "/home/kimlab1/strokach/working/pmc_tables/pmc_tables/pmc_tables.py", line 34, in save_archive_to_hdf5
    tables = parser(table_file)
  File "/home/kimlab1/strokach/working/pmc_tables/pmc_tables/parsers/_common.py", line 12, in wraped
    results = fn(filepath)
  File "/home/ki

In [378]:
!echo {DATA_DIR}.py

/home/kimlab1/strokach/datapkg/pmc-tables/notebooks/archive_to_hdf5/pmc_archives_0.py
